This notebook presents an interaction of Diva tools to read data with the [leaflet](http://leafletjs.com/) library using the python [folium](http://python-visualization.github.io/folium/) package.<be>
You will need to use the Diva read and plot functions available in the same directory, as well as the example input files.

In [ ]:
import os
import logging
import pydiva2d
import numpy as np
import matplotlib.pyplot as plt
import folium
from folium import plugins

In [ ]:
logger = logging.getLogger('diva2D')
logger.setLevel(logging.DEBUG)

# Get the data

The input files are located in the *data* directory.

In [ ]:
datadir = './data/'
datafile = os.path.join(datadir, 'MLD1.dat')
coastfile = os.path.join(datadir, 'coast.cont')
paramfile = os.path.join(datadir, 'param.par')

As in the previous example we create the directory and file names:

In [ ]:
divadir = "/home/ctroupin/Software/DIVA/diva-4.7.1"
DivaDirs = pydiva2d.DivaDirectories(divadir)
DivaFiles = pydiva2d.Diva2Dfiles(DivaDirs.diva2d)

Let's read the information from the various input and output files using the functions of the [diva2Dread](./diva2Dread.py) module.

In [ ]:
lon, lat, field = diva2Dread.read_data(datafile)
contours = diva2Dread.read_contour(coastfile)
xnode, ynode, i1, i2, i3 = diva2Dread.read_mesh(meshfile, meshtopofile)
lc, icoord, ispec, ireg, xori, yori, dx, dy, nx, ny, valex, snr, varbak = \
diva2Dread.read_parameters(paramfile)
x_analysis, y_analysis, analysis, error = diva2Dread.read_results(resultfile)

# Make the map

We will center the map on the mean location of the data, hoping that there are not bad data.<br>
For the zoom level, you need some tries to find the right level. It has to be an integer.

In [ ]:
divamap = folium.Map(location=[lat.mean(), lon.mean()], zoom_start=6)

## Add data points

We will also add a popup for each data marker, so that one can see the actual measurement at that location.

In [ ]:
for x, y, MLD in zip(lon, lat, field):
    folium.CircleMarker([y, x], popup="MLD = {0} m".format(np.round(MLD, 2))).add_to(divamap)

## Add contours

In [ ]:
for contour in contours:
    c = folium.PolyLine(locations=np.fliplr(np.append(contour, contour[0:1, :], axis=0)), 
                        color='orange', weight=5, opacity=0.75)
    divamap.add_child(c)
divamap

## Add finite element mesh

Due to the large number of nodes in the mesh, we prefer not to add them directory to the map.<br>
The solution is to first create a figure, then load it and add it to the map as an overlay. We have to remove all the axis, labels, frames and borders.

In [ ]:
fig = plt.figure(frameon=False)
ax = fig.add_axes([0, 0, 1, 1])
ax.axis('off')
f1 = plt.gca()
diva2Dplot.plot_mesh(xnode, ynode, i1, i2, i3, ax)
ax.set_xlim(xori, xori + nx * dx)
ax.set_ylim(yori, yori + ny * dy)
f1.axes.get_xaxis().set_ticks([])
f1.axes.get_yaxis().set_ticks([])
plt.savefig('./figures/BlackSeaMesh.png', transparent=True, 
            bbox_inches='tight', pad_inches=0)
plt.close()

The plugin [ImageOverlay](https://github.com/python-visualization/folium/blob/master/folium/plugins/image_overlay.py) is what we need to add the generated image.<br>
We specify the bounds using the information extracted from the parameter file.

In [ ]:
plugins.ImageOverlay(
    image=open('./figures/BlackSeaMesh.png', 'br'),
    bounds=[[yori, xori], [yori + ny * dy, xori + nx * dx]],
    opacity=0.8,
).add_to(divamap)
divamap

## Add analysed field

We follow the same approach as with the finite element mesh: first create the figure, then add it as an overlay.

In [ ]:
analysis = np.ma.masked_where(analysis==valex, analysis)
fig = plt.figure(frameon=False)
ax = fig.add_axes([0, 0, 1, 1])
diva2Dplot.plot_field(x_analysis, y_analysis, analysis)
ax.axis('off')
ax.set_xlim(xori, xori + nx * dx)
ax.set_ylim(yori, yori + ny * dy)
f1 = plt.gca()
f1.axes.get_xaxis().set_ticks([])
f1.axes.get_yaxis().set_ticks([])
plt.savefig('./figures/BlackSeaAnalysis.png', transparent=True, 
            bbox_inches='tight', pad_inches=0)
plt.close()

In [ ]:
plugins.ImageOverlay(
    image=open('./figures/BlackSeaAnalysis.png', 'br'),
    bounds=[[yori, xori], [yori + ny * dy, xori + nx * dx]],
    opacity=0.5,
).add_to(divamap)
divamap

Finally we export the map in html.

In [ ]:
divamap.save('diva_mesh_analysis.html')